In [1]:
import numpy as np
import pandas as pd 
import re

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/train.csv')
test_data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/test.csv')

In [3]:
data['tweet'] = data['tweet'].apply(lambda x: x.lower())
data['tweet'] = data['tweet'].apply((lambda x: re.sub(r"http\S+", '', x)))
data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
test_data['tweet'] = test_data['tweet'].apply(lambda x: x.lower())
test_data['tweet'] = test_data['tweet'].apply((lambda x: re.sub(r"http\S+", '', x)))
test_data['tweet'] = test_data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [7]:
max_features = 100
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['tweet'].values)
X = tokenizer.texts_to_sequences(data['tweet'].values)
X = pad_sequences(X)
Y = pd.get_dummies(data['handle']).values
X_test = tokenizer.texts_to_sequences(test_data['tweet'].values)
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size = 0.33, random_state = 42)
X_test = pad_sequences(X_test, X_train.shape[1])
print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)
print(X_test.shape, X.shape)

(3350, 21) (3350, 2)
(1650, 21) (1650, 2)
(1444, 21) (5000, 21)


In [8]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(64,activation='softmax'))# input_shape=(16,)))
#model.add(Dense(32,activation='softmax'))# input_shape=(16,)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 128)           12800     
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 21, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 267,994
Trainable params: 267,994
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 1)
score,acc = model.evaluate(X_val, Y_val, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

Epoch 1/7
3350/3350 [==============================] - 4s 1ms/step - loss: 0.5460 - acc: 0.7212
Epoch 2/7
3350/3350 [==============================] - 3s 924us/step - loss: 0.4020 - acc: 0.8218
Epoch 3/7
3350/3350 [==============================] - 3s 924us/step - loss: 0.3816 - acc: 0.8236
Epoch 4/7
3350/3350 [==============================] - 3s 933us/step - loss: 0.3664 - acc: 0.8328
Epoch 5/7
3350/3350 [==============================] - 3s 932us/step - loss: 0.3607 - acc: 0.8367
Epoch 6/7
3350/3350 [==============================] - 3s 931us/step - loss: 0.3473 - acc: 0.8487
Epoch 7/7
3350/3350 [==============================] - 3s 935us/step - loss: 0.3430 - acc: 0.8484
score: 0.40
acc: 0.82


In [ ]:
model.fit(X, Y, epochs = 7, batch_size=batch_size, verbose = 2)

In [ ]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = "/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/submission_"+str(timestr)+".csv"
with open(filename,"w+") as outputfile:
    outputfile.write("id,realDonaldTrump,HillaryClinton\n")
    for i in range(len(X_test)):
        hillary, donald = model.predict(X_test[i].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
        outputfile.write(str(i)+","+str(donald)+","+str(hillary)+"\n")